# Modeling

Now that we have explored our data, and cleaned it up into an acceptable format, we are ready to run some modeling techniques on it. We will begin by building multiple generic models on the data set to see how well it performs. After this, we will build a Neural Network to see if we can improve the performance from the baseline models.

In [40]:
#Import Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

### Loading Dataset

This dataset is already cleaned up and preprocessed. It is expected that this dataset has the following columns:
* SALEPRICE
* PROPERTYZIP

> NOTE/TODO: Currently, we don't do anything with the PROPERTYZIP. We plan to build a model for each zipcode, but that is pending our preliminary results (to see if we need to even attempt such a thing).

In [41]:
# Loading Data
data = pd.read_csv('output_total_df.csv')

# Renaming columns to not include brackets, spaces, or commas
column_mapping = {}
for col in list(data.columns):

    new_col = col.replace(']', ')')
    new_col = new_col.replace('[', '(')
    new_col = new_col.replace(', ', '-')
    new_col = new_col.replace(' ', '')
    column_mapping[col] = new_col

data = data.rename(columns=column_mapping)

# Drop Nulls
data = data.dropna()

In [42]:
data.head()

,Unnamed:0,STYLE,ROOF,EXTFINISH,BASEMENT,STORIES,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,FINISHEDLIVINGAREA,ZIPCODE,YEARSOLD,SALEDATE,SALEMONTH,PREVSALEPRICE,COUNTS,Condition,GRADE,CDU,LOTAREA,COUNTYTOTAL,COUNTYBUILDING,COUNTYLAND,HOOD,FAIRMARKETTOTAL,FAIRMARKETBUILDING,FAIRMARKETLAND,LOCALTOTAL,LOCALBUILDING,LOCALLAND,SALEPRICE
0,11,1.14236,0.60914,1.434605,4.059843,-1.264172,-2.140282,-2.394149,-0.763327,0.796003,-0.529069,0.66981,-1.831834,-1.190751,-0.869912,2.228069,0.856257,0.057441,-4.048448,4.0,-0.308887,1.501219,2.260656,-1.177967,-0.269532,1.364566,2.086445,-1.173908,1.366649,2.086445,-1.178272,394000.0
1,75,1.14236,0.60914,1.434605,4.059843,-1.264172,-1.537534,-1.237946,0.782665,-0.947308,-0.296914,0.66981,-1.831834,-0.788467,0.741401,-0.616165,1.198754,0.057441,-4.048448,4.0,-0.308887,2.326670,3.279850,-1.177967,-0.269532,2.179001,3.091708,-1.173908,2.182061,3.091708,-1.178272,400000.0
2,82,1.14236,0.60914,1.434605,4.059843,-1.264172,-2.140282,-2.394149,-0.763327,0.796003,-1.134329,0.66981,-1.831834,-1.238672,0.419138,1.099104,1.536139,0.057441,-4.048448,4.0,-0.308887,1.138164,1.812389,-1.177967,-0.269532,1.006357,1.644306,-1.173908,1.008011,1.644306,-1.178272,302000.0
3,126,1.14236,0.60914,1.434605,4.059843,-1.264172,-2.140282,-2.394149,0.782665,-0.947308,-0.516632,0.66981,-1.831834,-0.917097,0.096875,3.201099,0.314396,0.057441,-4.048448,4.0,-0.308887,2.085537,2.982120,-1.177967,-0.269532,2.101478,2.996021,-1.173908,2.104446,2.996021,-1.178272,425000.0
4,127,1.14236,0.60914,1.434605,4.059843,-1.264172,-1.537534,-1.237946,0.782665,-0.947308,-0.584344,0.66981,-1.831834,-0.784684,0.419138,-0.601270,2.088224,0.057441,-4.048448,4.0,-0.308887,1.432581,2.175909,-1.177967,-0.269532,1.457237,2.200830,-1.173908,1.459431,2.200830,-1.178272,300000.0


## Train/Validation/Test Split

We will split our dataset into three sets:
* Training - (77%)
* Validation - (16.5%)
* Testing - (16.5%)

In [43]:
from sklearn.model_selection import train_test_split

X = data.drop(['Unnamed:0', 'SALEPRICE'], axis=1)
Y = data['SALEPRICE']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, shuffle=True, random_state=100)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=100)

print("Training Shape: ", X_train.shape)
print("Validation Shape: ", X_val.shape)
print("Testing Shape: ", X_test.shape)

Training Shape:  (55336, 30)
Validation Shape:  (13628, 30)
Testing Shape:  (13628, 30)


In [44]:
X_train.head(1)

,STYLE,ROOF,EXTFINISH,BASEMENT,STORIES,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,FINISHEDLIVINGAREA,ZIPCODE,YEARSOLD,SALEDATE,SALEMONTH,PREVSALEPRICE,COUNTS,Condition,GRADE,CDU,LOTAREA,COUNTYTOTAL,COUNTYBUILDING,COUNTYLAND,HOOD,FAIRMARKETTOTAL,FAIRMARKETBUILDING,FAIRMARKETLAND,LOCALTOTAL,LOCALBUILDING,LOCALLAND
59976,-0.820377,-0.270608,-0.274254,-0.28344,-1.264172,-0.934785,-1.237946,0.782665,-0.947308,-0.356335,-0.07475,-0.560836,-0.724993,-0.869912,-0.616173,0.984055,-1.679064,-0.382411,3.0,0.504021,-0.135235,-0.180281,0.0385,0.308284,-0.089656,-0.123154,0.036462,-0.089316,-0.123154,0.038264


# Modeling
We will build some models on our data and see which model performs the best. The following models will be evaluated:

* XGBoost
* RandomForest
* KNearest Neighbors
* Support Vector Machine
* Gradient Boosted Decision Tree

## Testing Function
Here we will define a function that accepts a model, parameters, and data. This model will build the model and test it.

This function will be useful for testing all of our models.

In [45]:
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import time

def test_model(model, params, model_name, X_train, y_train, X_test, Y_test):
    """Trains model, and evaluates it.
        PARAMS:
            model_choice - SKLearn Model: Model to be trained
            params - dictionary: Dictionary of parameters to feed the model
            X_train - DataFrame: Training Data, Features
            y_train - DataFrame: Training Data, Targets
            X_test - DataFrame: Testing Data, Features
            y_test - DataFrame: Testing Data, Targets
        
        RETURNS:
            Mean Squared Log Error, Tree Explainer, Model Name - tuple(float, Explainer, String): Accuracy for specified model and parameters. 
    """
    print("Begin ", model_name)
    start = time.time()

    # Run model and get predictions and Mean Squared Log Error
    clf = model(**params)
    clf.fit(X_train, y_train)
    
    predictions = clf.predict(X_test)
    if predictions.min() < 10000:
        print("\t- [WARNING] - Out of bounds prediction... Replacing with 10,000")
        predictions[predictions < 0] = 10000
    
    msle = mean_squared_log_error(Y_test, predictions)
    rmse = mean_squared_error(Y_test, predictions)**0.5
    r2 = r2_score(Y_test, predictions)
    
    print(f"\t- MSLE: ", msle)
    print(f"\t- RMSE: ", rmse)
    print(f"\t- R^2: ", r2)
    
    print("\t- Time Elapsed: ", time.time() - start)
    
    return (msle, clf, model_name)

## Testing Models
We will run all of the following models through the testing function:
* XGBoost
* RandomForest
* KNearest Neighbors
* Support Vector Machine
* Gradient Boosted Decision Tree

Each of these models will be trained on the `Training Set` and evaluated on the `Validation Set`. The model that performs the best on the evaluation set (lowest `Mean Squared Log Error`), will be promoted to the next phase of model tuning (Hyper Parameter Tuning).

In [46]:
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

knc_params = {}
svr_params = {}
gbr_params = {}
xgb_params = {'objective':'reg:squarederror'}
rf_params = {}

models_to_test = [
    (xgb.XGBRegressor, xgb_params, "XGBoost"),
    (RandomForestRegressor, rf_params, "Random Forest"),
    (KNeighborsRegressor, knc_params, "KNeighbors"),
    (LinearSVR, svr_params, "Support Vector Machine"),
    (GradientBoostingRegressor, gbr_params, "GradientBoostingRegressor")
]

results = []
for model in models_to_test:
    results += [test_model(model[0], model[1], model[2], X_train, y_train, X_val, y_val)]

Begin  XGBoost


C:\Users\Matthew Tate\AppData\Roaming\Python\Python37\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


	- [WARNING] - Out of bounds prediction... Replacing with 10,000
	- MSLE:  0.05623469648617925
	- RMSE:  28832.66980219651
	- R^2:  0.9419770259830489
	- Time Elapsed:  5.906584978103638
Begin  Random Forest
	- MSLE:  0.04818997080637017
	- RMSE:  27866.560397262212
	- R^2:  0.9458002851712443
	- Time Elapsed:  67.4079041481018
Begin  KNeighbors
	- MSLE:  0.07997512394461233
	- RMSE:  36342.78702005015
	- R^2:  0.9078136274043938
	- Time Elapsed:  26.35189437866211
Begin  Support Vector Machine
	- [WARNING] - Out of bounds prediction... Replacing with 10,000
	- MSLE:  0.1100803927622709
	- RMSE:  44727.22823532089
	- R^2:  0.8603714462681604
	- Time Elapsed:  0.07252264022827148
Begin  GradientBoostingRegressor
	- [WARNING] - Out of bounds prediction... Replacing with 10,000
	- MSLE:  0.05773086701278188
	- RMSE:  29144.16668986349
	- R^2:  0.9407165386242988
	- Time Elapsed:  15.921813011169434


In [47]:
# Sort results by the best accuracy.
results.sort(key = lambda x: x[0], reverse=False)

print("Best Model: ", results[0][2])

Best Model:  Random Forest


## Hyper Parameter Tuning
`Random Forest` has the best performance.

We will now run a Grid Search on it to see which parameters might be most effective. We have already run a Randomized Grid Search (in the past) to see how we should narrow down our parameter space. 

    > NOTE: We are choosing to use a smaller `max_depth`. We do this so it can act as a regularization term. We may not get better results (than a higher max_depth), but we will get a more general model.

In [9]:
from sklearn.model_selection import GridSearchCV

def grid_search(clf, params, model_name, X_train, y_train, X_test, y_test):
    """Run Grid Search on a model"""
    print("Begin ", model_name)
    
    start = time.time()

    clf = GridSearchCV(clf(), params, refit=False, scoring='neg_mean_squared_log_error', cv=3, n_jobs=-1, verbose=10)
    clf.fit(X_train, y_train)
    
    print("\t- Best Score: ", clf.best_score_)
    print("\t- Time Elapsed: ", time.time() - start)
    
    return (clf.best_score_, clf, model_name)

params_rf = {
    'bootstrap': [True],
    'max_depth': [10,30,50,70], # Making this small to act as regularization.
    'max_features': ['auto',0.9,0.8],
    'min_samples_leaf': [1, 3],
    'min_samples_split': [4, 6],
    'n_estimators': [400,600,800]
}

models_to_tune = [
    (RandomForestRegressor, params_rf, "RandomForest")
]

tuned_results = []
for model in models_to_tune:
    tuned_results += [grid_search(model[0], model[1], model[2], X_train, y_train, X_val, y_val)]

Begin  RandomForest
Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 22.7min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed: 26.5min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed: 32.2min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed: 39.6min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed: 44.7min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed: 51.5min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed: 66.4min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 79.5min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed: 97.3min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 110.5min
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed: 

	- Best Score:  -0.07681857397299258
	- Time Elapsed:  19862.022178649902


## Best Parameters
We can view the best parameters by extracting `cv_results_`. This will allow us to see how the best parameters compared to the other parameters.

During the Hyper Parameter Tuning process, this was useful in order to narrow the search space down. For example, We saw consistently worse results during the following:
* bootstrap : False
* max_features : sq_rt

Armed with this knowledge, we were able to reduce the number of parameters we need to search over, so GridSearch might be more efficient

In [10]:
# Grab parameter results
param_results = pd.DataFrame(tuned_results[0][1].cv_results_).sort_values('rank_test_score')

# Write Parameters to CSV
import os
# if file does not exist write header 
if not os.path.isfile('random_forest_params.csv'):
    param_results.to_csv('random_forest_params.csv', header=True)
else: # else it exists so append without writing the header
    param_results.to_csv('random_forest_params.csv', mode='a', header=False)

param_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
83,774.786263,6.446918,7.233926,0.955843,True,50,auto,3,6,800,"{'bootstrap': True, 'max_depth': 50, 'max_feat...",-0.077626,-0.077542,-0.075287,-0.076819,0.001083,1
47,774.011335,3.330593,26.130245,5.434318,True,30,auto,3,6,800,"{'bootstrap': True, 'max_depth': 30, 'max_feat...",-0.077611,-0.077623,-0.075236,-0.076823,0.001122,2
73,656.206079,5.563231,7.214401,0.934333,True,50,auto,1,4,600,"{'bootstrap': True, 'max_depth': 50, 'max_feat...",-0.077769,-0.077457,-0.075251,-0.076826,0.001121,3
44,770.665531,1.546265,6.209767,0.058614,True,30,auto,3,4,800,"{'bootstrap': True, 'max_depth': 30, 'max_feat...",-0.077631,-0.077678,-0.075172,-0.076827,0.001170,4
46,575.294881,1.597125,4.497974,0.038676,True,30,auto,3,6,600,"{'bootstrap': True, 'max_depth': 30, 'max_feat...",-0.077564,-0.077639,-0.075282,-0.076828,0.001094,5


## Evaluate Model

We will evaluate the performance of the `RandomForestRegressor` model on the 
* Training Set
* Validation Set
* Testing Set

Understanding how the results varies within each of the data sets will help us understand how well the model is generalizing to new data. We expect to see a similar results across all data sets.

> NOTE: We will only using the results on the `testing` for determining to accept or decline our model. 

In [11]:
from sklearn.metrics import mean_squared_log_error

start = time.time()

best_clf = RandomForestRegressor(**tuned_results[0][1].best_params_).fit(X_train, y_train)

print("Time Elapsed: ", time.time() - start)

predictions = best_clf.predict(X_train)
msle = mean_squared_log_error(y_train, predictions)
print("Training Mean Squared Log Error: ", msle)

predictions = best_clf.predict(X_val)
msle = mean_squared_log_error(y_val, predictions)
print("Validation Mean Squared Log Error: ", msle)

predictions = best_clf.predict(X_test)
msle = mean_squared_log_error(y_test, predictions)
print("Testing Mean Squared Log Error: ", msle)

Time Elapsed:  747.2535169124603
Training Mean Squared Log Error:  0.028079280506422465
Validation Mean Squared Log Error:  0.07418327885404077
Testing Mean Squared Log Error:  0.07684279699857259


## Zillow's Zestimate
Zillow presents their results via a table that displays the following:
* Median Error
* Homes With ZESTIMATES
* Within 5% of Sale Price
* Within 10% of Sale Price
* Within 20% of Sale Price

We will develop the same metrics from our results and compare them to Zillows results on `Pittsburgh PA`

In [12]:
y_test = y_test.reset_index(drop=True)

# Concatentate Predictions with Actual
comparison = pd.concat([pd.Series(predictions), y_test], axis=1)
comparison = comparison.rename(columns={0:'PREDICTION'})

# Calculate `Percent Error`
## |(Prediction - SalePrice) / SalePrice|
comparison['PERCENT ERROR'] = abs((comparison['PREDICTION'] - comparison['SALEPRICE']) / comparison['SALEPRICE'])

# Write to CSV
comparison.to_csv('predictions.csv')

In [13]:
comparison.head()

,PREDICTION,SALEPRICE,PERCENT ERROR
0,53860.034574,63000.0,0.145079
1,47787.256382,51000.0,0.062995
2,125579.718215,115000.0,0.091998
3,344395.786402,378000.0,0.088900
4,132732.113023,146000.0,0.090876


In [14]:
# Obtain Metrics to create table.

area = 'Pittsburgh, PA'
median_error = comparison['PERCENT ERROR'].median() * 100
total_estimates = comparison.shape[0]
five_percent = comparison[comparison['PERCENT ERROR'] <= 0.05].shape[0] / comparison.shape[0] * 100
ten_percent = comparison[comparison['PERCENT ERROR'] <= 0.10].shape[0] / comparison.shape[0] * 100
twenty_percent = comparison[comparison['PERCENT ERROR'] <= 0.20].shape[0] / comparison.shape[0] * 100

column_names = ['METROPOLITAN AREA', 'MEDIAN ERROR', 'HOMES WITH ESTIMATE', 'WITH 5% OF SALEPRICE', 'WITH 10% OF SALEPRICE', 'WITH 20% OF SALEPRICE']

results_lst = [area, median_error, total_estimates, five_percent, ten_percent, twenty_percent]
zillow_lst = ['Pittsburgh, PA', 2.5, 10900, 77.6, 93.2, 98.4]

results = pd.DataFrame([results_lst, zillow_lst], index=['Our Results', 'Zillow'], columns=column_names)

results.head()

,METROPOLITAN AREA,MEDIAN ERROR,HOMES WITH ESTIMATE,WITH 5% OF SALEPRICE,WITH 10% OF SALEPRICE,WITH 20% OF SALEPRICE
Our Results,"Pittsburgh, PA",10.409506,23704,27.62403,48.439082,73.202835
Zillow,"Pittsburgh, PA",2.500000,10900,77.60000,93.200000,98.400000


# Feature Importance:

### Random Forest Feature Importance
Random Forest has a built in feature importance, which will tell us how important each feature is. However, this does not tell us what kind of values the features should have.


### SHAP (SHapley Additive exPlanations)
SHAP measures the impact of variables taking into account the interaction with other variables. 

This will be better than the feature importances found in Random Forest because we can now see how the value of each feature affects the final result.

Reference: 
* https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d

* https://blog.datascienceheroes.com/how-to-interpret-shap-values-in-r/Which features are more important to XGBoost?

In [31]:
features = X.columns.tolist()
importances = results[0][1].feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

KeyError: 0

In [ ]:
import shap
print(shap.__version__)

# May need to run the following as system admin in anaconda command prompt
## conda install -c conda-forge shap

In [ ]:
shap_values = shap.TreeExplainer(results[0][1]).shap_values(X_train)

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_train) # Change to Red and Green

## Results

We tested the following models:
* XGBoost
* RandomForest
* KNearest Neighbors
* Support Vector Machine
* Gradient Boosted Decision Tree

The best performing model was `RandomForest`. Once we found the best performing model, we implemented a `Grid Search` which aimed to exhaustively explore a specified grid and find the best parameters. Once we tuned our parameters, we tested our predictions on our Training, Validation, and Testing set.

Our testing set had a RMSE of `4780.84`. This indicates that we still have a lot of room for improvements, so we might want to attempt a different approach.